In [11]:
from cmp import *
% matplotlib notebook
np.set_printoptions(threshold=np.nan)
import pdir

In [33]:
def plane_limiter(z, r_min, r_max):
    """
    Limiter function for the planes, as they have a different data structure to
    the list of points.
    """
    # We take z and compare each value to the r_min[2] and r_max[2]. If it is outside the range, then we replace the value with nan
    outside = (r_min[2] > z) + (z > r_max[2])
    z[outside] = np.nan
    return z


def null_space(G):
    """
    creates two unit vectors that are perpendicular to each other and to G
    """
    v1 = np.array([1,0,0])
    cosG1 = G.dot(v1)/(lattices.mag(G))
    if eq(cosG1, 1) or eq(cosG1, -1):
        # G is along x-axis. We choose y and z as v1 and v2:
        return np.array([0,1,0]), np.array([0,0,1])
    
    # If we haven't returned the function, then we need to do some further manipulation.
    # First we rotate v1 such that it is perpendicular to G, and we do this along a vector perpendicular to both (which we might as well use for v2):
    v2 = np.cross(G, v1)
    v2 = v2/lattices.mag(v2)
    alpha = np.arccos(cosG1)
    theta = np.pi/2-alpha
    
    R = lattices.rot_matrix(v2, theta)
    v1 = R @ v1
    
    cosG1 = G.dot(v1)/(lattices.mag(G))
    if not eq(cosG1, 0):
        R = lattices.rot_matrix(v2, -2*theta)
        v1 = R @ v1
    return v1, v2


def reciprocal(a1, a2, a3, h, k, l, range_ = (-20,20)):
    """
    Creates the reciprocal lattice and a given family of lattice planes.
    """
    
    # First the scaling factor for the reciprocal lattice
    scale = a1.dot(np.cross(a2, a3))
    # Then the reciprocal lattice
    b1 = 2 * np.pi * np.cross(a2, a3) / scale
    b2 = 2 * np.pi * np.cross(a3, a1) / scale
    b3 = 2 * np.pi * np.cross(a1, a2) / scale
    
    # And the normal vector for the (hkl)-family of planes.
    G = h * b1 + k * b2 + l * b3
    v1, v2 = null_space(G)
    G_unit = G / lattices.mag(G)
    
    # Next the displacement vector d
    d = 2 * np.pi * G_unit / lattices.mag(G)

    P, Q = np.meshgrid(range(-5, 6), range(-5, 6))

    # Now the starting plane
    x0 = v1[0]*P + v2[0]*Q
    y0 = v1[1]*P + v2[1]*Q
    z0 = v1[2]*P + v2[2]*Q

    # Create a list of the planes with a list comprehension
    planes = [(x0 + n*d[0], y0 + n*d[1], z0 + n*d[2]) for n in range(range_[0], range_[1])]

    return d, planes


def Reciprocal(a1=d[0], a2=d[1], a3=d[2], indices=np.array([1, 1, 1]),
               colors=d[4], sizes=d[5],
               min_=d[8], max_=d[9], lattice_name=None, verbose=False):
    if lattice_name is not None:
        lattice, basis, lattice_type = lattices.chooser(lattice_name, verbose=verbose)
        a1, a2, a3 = lattice
        # Classify the lattice
    else:
        lattice_type = lattices.classifier(a1, a2, a3, basis)

        # Rotate the lattice
        a1, a2, a3, basis = lattices.rotator(a1, a2, a3, basis,
                                             lattice_type, verbose=verbose)
    
    basis = np.array([0,0,0])
    
    length_basis = np.shape(basis)
    if len(length_basis) == 1:
        n_basis = 1
    elif len(length_basis) > 1:
        n_basis = length_basis[0]

    # Make a list, n_basis long, for the colors and sizes,
    # if they're not specified.
    c_name = colors.__class__.__name__
    if c_name == "str":
        c = colors
        colors = []
        for i in range(n_basis):
            colors.append(c)
    elif c_name == "list" and len(colors) < n_basis:
        c = colors[0]
        colors = []
        for i in range(n_basis):
            colors.append(c)

    s_name = sizes.__class__.__name__
    if s_name == "int" or s_name == "float":
        s = sizes
        sizes = []
        for i in range(n_basis):
            sizes.append(s)
    elif s_name == "list" and len(sizes) < n_basis:
        s = sizes[0]
        sizes = []
        for i in range(n_basis):
            sizes.append(s)
    
    # Next we find the limits:
    lim_type = 'proper'
    r_min, r_max, n_min, n_max = lattices.find_limits(lim_type, a1, a2, a3,
                                                      min_, max_)
    
    (atomic_positions, lattice_coefficients, atomic_colors, atomic_sizes,
     lattice_position) = lattices.generator(a1, a2, a3, basis, colors, sizes,
                                            lim_type, n_min, n_max, r_min,
                                            r_max)
    # Then we get the planes
    h,k,l = indices
    d, planes = reciprocal(a1, a2, a3, h, k, l, range_=[0,3])
    
    
    # Objects to limit to the plot-box
    objects = [atomic_positions, lattice_coefficients, atomic_colors,
               atomic_sizes, lattice_position]
    objects = lattices.limiter(atomic_positions, objects, r_min, r_max)
    (atomic_positions, lattice_coefficients, atomic_colors, atomic_sizes,
     lattice_position) = objects
    
    # Prune each of the planes
    # planes = [plane_limiter(p, r_min, r_max) for p in planes]
    # Create the figure
    fig = plt.figure()
    ax = fig.gca(projection="3d")

    # Plot atoms. For now a single size and color
    ax.scatter(atomic_positions[:, 0], atomic_positions[:, 1],
               atomic_positions[:, 2], c=atomic_colors, s=atomic_sizes)

    # plot the displacement vector
    ax.quiver(0, 0, 0, d[0], d[1], d[2])
    ax.text(d[0] / 2, d[1] / 2, d[2] / 2, '$d$')

    for p in planes:
        ax.plot_surface(p[0], p[1], p[2], color='xkcd:cement', shade=False, alpha = 0.4)
    # Set limits, orthographic projection (so we get the beautiful hexagons),
    # no automatic gridlines, and no axes
    ax.set_aspect('equal')
    ax.set_proj_type('ortho')
    ax.set_xlim3d([r_min[0], r_max[0]])
    ax.set_ylim3d([r_min[1], r_max[1]])
    ax.set_zlim3d([r_min[2], r_max[2]])
    ax.grid(False)
    plt.axis('equal')
    plt.axis('off')

    # make the panes transparent (the plot box)
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    plt.show()

#Reciprocal(lattice_name = "simple cubic", indices = (1,1,1))

In [4]:
# Inputs
eq = np.isclose
# Lattice vectors (3 vectors of length 3)
a = 1
b = 2
a1 = np.array([1, 0, 0])
a2 = np.array([0, 1, 0])
a3 = np.array([0, 0, 1])
theta = 80*np.pi/180

# Array of basis vectors
basis = np.array([[0,0,0],[0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]])
# Colors for each of the basis vectors
blargh = ('r', 'r','b','b')
# Size multiplier for each of the atoms. Default is 1
sizes = (2,2,1,1)
verbose = True


# Gridline type:
# Soft: Lines along cartesian axes. Takes into account nonequal lattice spacing
# LatticeVectors: Lines along the latticevectors (only on lattice points)
GridType = "lattice"

# Limit type:
# individual: Sets the limits as max(nx*a1,ny*a2,nz*a3), so we include nx unitcells in the a1 direction, etc.
# sum: Sets the limits r_min = n_min*[a1 a2 a3] and likewise for n_max
LimType = "dynamic"
Maxs = [2,2,2]
Mins = [0,0,0]

LatticeType = "simple cubic"

#Lattice(lattice_name = LatticeType, colors = blargh, sizes = sizes, max_ = Maxs, verbose=True)
